In [ ]:
import os
# Auto-generated setup for portability
if 'google.colab' in str(get_ipython()):
    # Assume data is mounted or downloaded to current dir in Colab
    BASE_DIR = os.getcwd()
else:
    # Local execution
    BASE_DIR = os.getcwd()


In [ ]:
import os
import json
import time
import random
from groq import Groq, RateLimitError, APIError
from tqdm import tqdm
import logging

# ======================
# CONFIGURATION
# ======================
# 🔑 Replace with your real key (or use os.getenv("GROQ_API_KEY"))
GROQ_API_KEY = os.getenv("GROQ_API_KEY", "xxxxxxxxxxxxxxxxxxxx")
if GROQ_API_KEY == "your_actual_api_key_here" or not GROQ_API_KEY:
    print("❌ Please set your GROQ_API_KEY environment variable or replace this value with your real API key!")
    print("💡 You can set it with: export GROQ_API_KEY='your_actual_key_here'")
    # Instead of raising an error, let's allow the user to set it interactively
    import getpass
    GROQ_API_KEY = getpass.getpass("🔑 Enter your Groq API key (or press Enter to skip): ").strip()
    if not GROQ_API_KEY:
        print("⚠️  No API key provided. The script will run but API calls will fail.")
        GROQ_API_KEY = "demo_key_that_will_fail"

MODEL_NAME = "meta-llama/llama-4-scout-17b-16e-instruct"
OUTPUT_FILE = "code_finetune_dataset.jsonl"
TARGET_TOTAL_EXAMPLES = 1000  # 🎯 The desired total number of examples
FLUSH_EVERY = 10  # Save to disk every N examples
LOG_FILE = "generation_log.log"

# Rate Limiting Configuration
BASE_DELAY = 1.5  # Base delay in seconds between requests (increased from 1.2)
MAX_DELAY = 60      # Maximum delay in seconds
BACKOFF_FACTOR = 1.5 # Factor by which delay increases after each failure

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(LOG_FILE),
        logging.StreamHandler()  # Also print to console
    ]
)

# 🧠 EXPLICIT PROMPT: tells the model *exactly* what to do
GENERATOR_SYSTEM_PROMPT = """You are an elite programming assistant. Generate a perfect training example in the following strict format:
<thinking>
- Restate the problem clearly.
- Compare at least two concrete implementation approaches (e.g., "use a hash map" vs "sort first").
- Analyze time and space complexity (Big O) for each.
- Choose the best approach and justify why.
- List 2–3 key edge cases (e.g., empty input, duplicates, None).
- Do NOT write code here.
</thinking>
<answer>
```python
def your_function(...) -> ...:
    \"\"\"
    Clear docstring: what it does, args, return.
    \"\"\"
    # Clean, efficient, well-commented code
    ...
</answer>
Rules:
    ALWAYS use this exact XML-like format.
    NEVER omit <thinking> or <answer>.
    Code must be valid, runnable Python with type hints.
    NO markdown outside the ```python block.
    End with </answer>.
    """

CODE_TOPICS = [
    # Data Structures
    "Implement a stack using only queues.",
    "Create a doubly linked list with insert, delete, and reverse methods.",
    "Write a function to flatten a nested dictionary.",
    "Implement an LRU cache with get and put operations.",
    "Design a circular buffer (ring buffer) in Python.",
    "Write a function to deep copy a nested list of dictionaries.",
    "Implement a trie (prefix tree) for word storage and search.",
    "Create a class for a priority queue using heapq.",
    # Strings & Text
    "Write a function to remove all duplicate characters from a string while preserving order.",
    "Implement a function to convert a sentence to title case without using built-in methods.",
    "Create a function that finds the longest common prefix among a list of strings.",
    "Write a function to replace all spaces in a string with '%20'.",
    "Implement a function to check if a string is a valid IPv4 address.",
    "Create a function that splits a camelCase string into words.",
    "Write a function to compress a string using run-length encoding.",
    "Implement a function to reverse words in a sentence without reversing letters.",
    # Files & I/O
    "Write a function to read a CSV file and return it as a list of dictionaries.",
    "Create a function that merges multiple JSON files into one.",
    "Implement a function to find all files with a given extension in a directory tree.",
    "Write a function to safely rename all files in a folder with a timestamp prefix.",
    "Create a context manager that times how long a block of code takes to run.",
    "Implement a function to backup a file by appending '.bak' and a timestamp.",
    # Web & APIs
    "Write a function to fetch JSON data from a REST API with error handling.",
    "Create a function that validates a URL using regex and urllib.",
    "Implement a simple rate-limited API client using time.sleep.",
    "Write a function to extract all email addresses from a webpage HTML string.",
    "Create a function that posts form data to a webhook with retries.",
    # Testing & Debugging
    "Write a decorator that logs function calls and execution time.",
    "Implement a function to generate a random password with configurable rules.",
    "Create a function that validates a Python dict against a schema.",
    "Write a function to compare two JSON objects for equality, ignoring key order.",
    # Concurrency
    "Write a function that downloads multiple URLs concurrently using threading.",
    "Create a thread-safe counter class using locks.",
    "Implement a producer-consumer queue using Python's queue module.",
    # Utilities
    "Write a function to convert seconds into a human-readable duration string.",
    "Create a function that parses a command-line argument string into a dict.",
    "Implement a function to find the most frequent word in a text file.",
    "Write a function to generate a UUID v4 without using the uuid module.",
    "Create a function that validates a credit card number using the Luhn algorithm.",
    "Implement a function to parse a log file and count error occurrences.",
    # OOP & Design
    "Create a singleton class in Python.",
    "Implement the observer pattern for a simple event system.",
    "Write a class that represents a bank account with deposit/withdraw methods.",
    "Create a plugin system using abstract base classes.",
    # More Practical Tasks
    "Write a function to sanitize user input for SQL queries (basic escaping).",
    "Implement a function to convert a Python dict to a URL query string.",
    "Create a function that detects if two rectangles overlap (given as (x1,y1,x2,y2)).",
    "Write a function to parse a date string in multiple formats and return a datetime.",
    "Implement a function to calculate the difference between two dates in human terms.",
    "Create a function that validates a phone number in E.164 format.",
    "Write a function to extract the domain from an email address.",
    "Implement a function to generate a slug from a title string.",
    "Create a function that checks if a file is binary or text.",
    "Write a function to rotate a 2D matrix 90 degrees clockwise in-place.",
    "Implement a function to find the intersection of two lists without duplicates.",
    "Create a function that batches a list into chunks of size N.",
    "Write a function to validate a JSON Web Token (JWT) signature (mock implementation).",
    "Implement a function to convert a list of dicts to a pandas-like table string.",
    "Create a function that finds the first non-repeating character in a string.",
    "Write a function to implement a simple state machine for parsing commands.",
    "Implement a function to calculate the checksum of a file.",
    "Create a function that converts a snake_case string to camelCase.",
    "Write a function to parse a simple INI config file into a dictionary.",
    "Implement a function to find all anagrams of a word in a given word list.",
    "Create a function that validates a username against common rules.",
    "Write a function to implement a basic spell checker using edit distance.",
    "Implement a function to generate a Fibonacci sequence as a generator.",
    "Create a function that merges overlapping intervals.",
    "Write a function to detect if a linked list has a cycle using Floyd's algorithm.",
    "Implement a function to serialize and deserialize a binary tree.",
    "Create a function that finds the kth largest element in an unsorted list.",
    "Write a function to implement a simple key-value store with TTL (time-to-live).",
    "Implement a function to parse a simple arithmetic expression (e.g., '2+3*4').",
    "Create a function that validates a Sudoku board.",
    "Write a function to implement a basic breadth-first search on a grid.",
    "Implement a function to find the shortest path in a maze using BFS.",
    "Create a function that converts a decimal number to binary without bin().",
    "Write a function to implement a simple in-memory message queue.",
    "Implement a function to calculate the median of a stream of numbers.",
    "Create a function that validates a YAML-like indentation structure.",
    "Write a function to implement a basic autocomplete using a trie.",
    "Implement a function to find all subsets of a set (power set).",
    "Create a function that validates a CSS color hex code.",
    "Write a function to implement a simple template engine with {{variable}} substitution.",
    "Implement a function to parse a command like 'ls -l /home' into args and options.",
    "Create a function that detects memory leaks by tracking object counts (mock).",
    "Write a function to implement a basic rate limiter using a token bucket.",
    "Implement a function to convert a Python dict to a JSON-serializable dict.",
    "Create a function that finds the longest palindromic substring.",
    "Write a function to implement a simple event emitter/listener pattern.",
    "Implement a function to validate a JWT payload expiration (mock).",
    "Create a function that converts a list of objects to CSV format.",
    "Write a function to implement a basic cache decorator with TTL.",
    "Implement a function to find the minimum window substring.",
    "Create a function that validates a cron expression (simplified).",
    "Write a function to implement a simple dependency resolver.",
    "Implement a function to calculate the edit distance between two strings.",
    "Create a function that parses a simple query string like 'name=John&age=30'.",
    "Write a function to implement a basic circuit breaker pattern.",
    "Implement a function to find the top K frequent elements in a list.",
    "Create a function that validates a file path for security (no '../').",
    "Write a function to implement a simple bloom filter.",
    "Implement a function to convert a nested dict to a flat dict with dot notation keys.",
    "Create a function that finds the longest increasing subsequence.",
    "Write a function to implement a simple object pool pattern.",
    "Implement a function to validate a GraphQL query structure (mock).",
    "Create a function that converts a list of tuples to a dictionary.",
    "Write a function to implement a simple state machine for order processing.",
    "Implement a function to find the maximum product subarray.",
    "Create a function that validates a Dockerfile instruction (mock).",
    "Write a function to implement a basic retry mechanism with exponential backoff.",
    "Implement a function to convert a Python enum to a list of choices.",
    "Create a function that finds the shortest unique prefix for each word.",
    "Write a function to implement a simple feature flag system.",
    "Implement a function to validate a Kubernetes YAML manifest (mock).",
    "Create a function that converts a list of objects to a markdown table.",
    "Write a function to implement a basic circuit breaker with failure threshold.",
    "Implement a function to find the maximum sum path in a triangle.",
    "Create a function that validates a Prometheus metric name.",
    "Write a function to implement a simple rate limiter using sliding window.",
    "Implement a function to convert a nested JSON to a flat JSON with path keys.",
    "Create a function that finds the longest common subsequence.",
    "Write a function to implement a basic object validator with rules.",
    "Implement a function to validate a Terraform HCL block (mock).",
    "Create a function that converts a list of objects to an Excel file (mock with openpyxl).",
    "Write a function to implement a simple service registry pattern.",
    "Implement a function to find the minimum number of coins for a given amount.",
    "Create a function that validates a CloudFormation template (mock).",
    "Write a function to implement a basic circuit breaker with half-open state.",
    "Implement a function to convert a Python dataclass to a JSON schema.",
    "Create a function that finds the maximum area of a histogram rectangle.",
    "Write a function to implement a simple feature toggle with environment variables.",
    "Implement a function to validate an OpenAPI spec operation (mock).",
    "Create a function that converts a list of objects to a CSV string.",
    "Write a function to implement a basic cache decorator with TTL.",
    "Implement a function to find the minimum window substring.",
    "Create a function that validates a cron expression (simplified).",
    "Write a function to implement a simple dependency resolver.",
    "Implement a function to calculate the edit distance between two strings.",
    "Create a function that parses a simple query string like 'name=John&age=30'.",
    "Write a function to implement a basic circuit breaker pattern.",
    "Implement a function to find the top K frequent elements in a list.",
    "Create a function that validates a file path for security (no '../').",
    "Write a function to implement a simple bloom filter.",
    "Implement a function to convert a nested dict to a flat dict with dot notation keys.",
    "Create a function that finds the longest increasing subsequence.",
    "Write a function to implement a simple object pool pattern.",
    "Implement a function to validate a GraphQL query structure (mock).",
    "Create a function that converts a list of tuples to a dictionary.",
    "Write a function to implement a simple state machine for order processing.",
    "Implement a function to find the maximum product subarray.",
    "Create a function that validates a Dockerfile instruction (mock).",
    "Write a function to implement a basic retry mechanism with exponential backoff.",
    "Implement a function to convert a Python enum to a list of choices.",
    "Create a function that finds the shortest unique prefix for each word.",
    "Write a function to implement a simple feature flag system.",
    "Implement a function to validate a Kubernetes YAML manifest (mock).",
    "Create a function that converts a list of objects to a markdown table.",
    "Write a function to implement a basic circuit breaker with failure threshold.",
    "Implement a function to find the maximum sum path in a triangle.",
    "Create a function that validates a Prometheus metric name.",
    "Write a function to implement a simple rate limiter using sliding window.",
    "Implement a function to convert a nested JSON to a flat JSON with path keys.",
    "Create a function that finds the longest common subsequence.",
    "Write a function to implement a basic object validator with rules.",
    "Implement a function to validate a Terraform HCL block (mock).",
    "Create a function that converts a list of objects to an Excel file (mock with openpyxl).",
    "Write a function to implement a simple service registry pattern.",
    "Implement a function to find the minimum number of coins for a given amount.",
    "Create a function that validates a CloudFormation template (mock).",
    "Write a function to implement a basic circuit breaker with half-open state.",
    "Implement a function to convert a Python dataclass to a JSON schema.",
    "Create a function that finds the maximum area of a histogram rectangle.",
    "Write a function to implement a simple feature toggle with environment variables.",
    "Implement a function to validate an OpenAPI spec operation (mock).",
    "Create a function that converts a list of objects to a CSV string.",
    "Write a function to implement a basic retry mechanism with circuit breaker.",
    "Implement a function to find the longest palindromic subsequence.",
    "Create a function that validates a Helm chart values file (mock).",
    "Write a function to implement a simple object pool for database connections.",
    "Implement a function to convert a nested dict to a protobuf-like message (mock).",
    "Create a function that finds the maximum sum of a contiguous subarray.",
    "Write a function to implement a basic rate limiter using Redis (mock).",
    "Implement a function to validate a GitHub Actions workflow (mock).",
    "Create a function that converts a list of objects to a JSON Lines string.",
    "Write a function to implement a simple circuit breaker with metrics.",
    "Implement a function to find the minimum path sum in a grid.",
    "Create a function that validates a Docker Compose service (mock).",
    "Write a function to implement a basic retry mechanism with circuit breaker.",
    "Implement a function to convert a Python class to a TypeScript interface (mock).",
    "Create a function that finds the maximum profit from stock prices.",
    "Write a function to implement a simple feature flag with user targeting.",
    "Implement a function to validate a Kubernetes deployment spec (mock).",
    "Create a function that converts a list of objects to a Parquet file (mock).",
    "Write a function to implement a basic object validator with custom rules.",
    "Implement a function to find the longest substring with at most K distinct characters.",
    "Create a function that validates a Terraform variable block (mock).",
    "Write a function to implement a simple service discovery pattern.",
    "Implement a function to convert a nested JSON to a GraphQL query (mock).",
    "Create a function that finds the maximum sum of a non-adjacent subarray.",
    "Write a function to implement a basic rate limiter with distributed lock (mock).",
    "Implement a function to validate a CloudFormation resource (mock).",
    "Create a function that converts a list of objects to an Avro schema (mock).",
    "Write a function to implement a simple circuit breaker with logging.",
    "Implement a function to find the minimum number of jumps to reach end.",
    "Create a function that validates a Prometheus alert rule (mock).",
    "Write a function to implement a basic retry mechanism with circuit breaker and logging.",
    "Implement a function to convert a Python function to a CLI command (mock with argparse).",
    "Create a function that finds the maximum sum of a submatrix.",
    "Write a function to implement a simple feature flag with A/B testing.",
    "Implement a function to validate a GitHub repository settings (mock).",
    "Create a function that converts a list of objects to a BigQuery schema (mock).",
    "Write a function to implement a basic object pool with health checks.",
    "Implement a function to find the longest common substring.",
    "Create a function that validates a Docker image tag (mock).",
    "Write a function to implement a simple rate limiter with token bucket and sliding window.",
    "Implement a function to find the maximum sum of a path in a binary tree.",
    "Write a function to implement a basic circuit breaker with fallback function.",
    "Implement a function to validate a Kubernetes ingress spec (mock).",
    "Create a function that converts a list of objects to a Firestore collection (mock).",
    "Write a function to implement a simple retry mechanism with exponential backoff and jitter.",
    "Implement a function to find the minimum window that contains all characters of a string.",
    "Create a function that validates a Helm chart template (mock).",
    "Write a function to implement a basic object validator with nested rules.",
    "Implement a function to convert a Python class to a JSON schema with OpenAPI extensions.",
    "Create a function that finds the maximum sum of a subarray with length at least K.",
    "Write a function to implement a simple feature flag with percentage rollout.",
    "Implement a function to validate a Terraform provider block (mock).",
    "Create a function that converts a list of objects to a Snowflake table DDL (mock).",
    "Write a function to implement a basic rate limiter with Redis and Lua script (mock).",
    "Implement a function to find the longest increasing subsequence with binary search.",
    "Create a function that validates a Docker network config (mock).",
    "Write a function to implement a simple circuit breaker with metrics and logging.",
    "Implement a function to convert a nested JSON to a Protobuf message (mock).",
    "Create a function that finds the maximum sum of a subarray with no two elements adjacent.",
    "Write a function to implement a basic retry mechanism with circuit breaker, logging, and metrics.",
    "Implement a function to validate a CloudFormation parameter (mock).",
    "Create a function that converts a list of objects to a Delta Lake table (mock).",
    "Write a function to implement a simple feature flag with user attributes.",
    "Implement a function to find the minimum number of operations to make array equal.",
    "Create a function that validates a Prometheus recording rule (mock).",
    "Write a function to implement a basic object pool with connection reuse.",
    "Implement a function to convert a Python dataclass to a GraphQL type (mock).",
    "Create a function that finds the maximum sum of a subarray with exactly K elements.",
    "Write a function to implement a simple rate limiter with distributed token bucket.",
    "Implement a function to validate a GitHub Actions job (mock).",
    "Create a function that converts a list of objects to a Cassandra table schema (mock).",
    "Write a function to implement a basic circuit breaker with half-open state and metrics.",
    "Implement a function to find the longest palindromic substring with Manacher's algorithm (simplified).",
    "Create a function that validates a Docker volume config (mock).",
    "Write a function to implement a simple retry mechanism with circuit breaker and exponential backoff.",
    "Implement a function to convert a nested dict to a YAML string with comments (mock).",
    "Create a function that finds the maximum sum of a subarray with at most K elements.",
    "Write a function to implement a simple feature flag with environment and user targeting.",
    "Implement a function to validate a Kubernetes ingress spec (mock).",
    "Create a function that converts a list of objects to a Bigtable schema (mock).",
    "Write a function to implement a simple object validator with custom error messages.",
    "Implement a function to find the minimum number of coins to make change with dynamic programming.",
    "Create a function that validates a Terraform output block (mock).",
    "Write a function to implement a basic rate limiter with sliding window and Redis.",
    "Implement a function to convert a Python function to a FastAPI endpoint (mock).",
    "Create a function that finds the maximum sum of a subarray with no duplicates.",
    "Write a function to implement a basic circuit breaker with fallback and logging.",
    "Implement a function to validate a CloudFormation mapping (mock).",
    "Create a function that converts a list of objects to a DynamoDB table schema (mock).",
    "Write a function to implement a simple retry mechanism with jitter and circuit breaker.",
    "Implement a function to find the longest common subsequence with space optimization.",
    "Create a function that validates a Docker secret config (mock).",
    "Write a function to implement a simple feature flag with A/B testing and metrics.",
    "Implement a function to convert a nested JSON to a Avro schema (mock).",
    "Create a function that finds the maximum sum of a subarray with at least K elements.",
    "Write a function to implement a basic rate limiter with token bucket and Redis.",
    "Implement a function to validate a GitHub Actions workflow job (mock).",
    "Create a function that converts a list of objects to a Redshift table DDL (mock).",
    "Write a function to implement a simple circuit breaker with metrics, logging, and half-open state.",
    "Implement a function to find the minimum number of operations to sort an array.",
    "Create a function that validates a Kubernetes configmap spec (mock).",
    "Write a function to implement a basic object validator with nested rules and custom errors.",
    "Implement a function to convert a Python class to a OpenAPI schema (mock).",
    "Create a function that finds the maximum sum of a subarray with exactly K distinct elements.",
    "Write a function to implement a simple rate limiter with token bucket, sliding window, and Redis.",
    "Implement a function to validate a GitHub Actions workflow trigger (mock).",
    "Create a function that converts a list of objects to a PostgreSQL table DDL (mock).",
    "Write a function to implement a basic circuit breaker with all features: fallback, logging, metrics, half-open state, and error threshold.",
    "Implement a function to find the longest common subsequence with reconstruction.",
    "Create a function that validates a Docker healthcheck config (mock).",
    "Write a function to implement a simple retry mechanism with all features: exponential backoff, jitter, circuit breaker, and logging.",
    "Implement a function to convert a nested JSON to a GraphQL schema (mock).",
    "Create a function that finds the maximum sum of a subarray with constraints on element values.",
    "Write a function to implement a basic feature flag with all features: A/B testing, percentage rollout, user targeting, and environment rules.",
    "Implement a function to validate a Terraform resource block (mock).",
    "Create a function that converts a list of objects to a MySQL table DDL (mock).",
    "Write a function to implement a simple rate limiter with all features: token bucket, sliding window, Redis, and distributed lock.",
    "Implement a function to find the minimum number of operations to transform one string to another.",
    "Create a function that validates a Kubernetes pod spec (mock).",
    "Write a function to implement a basic object validator with all features: nested rules, custom errors, type checking, and schema validation.",
    "Implement a function to convert a Python class to a JSON Schema with all OpenAPI extensions (mock).",
    "Create a function that finds the maximum sum of a subarray with dynamic constraints.",
    "Write a function to implement a simple circuit breaker with maximum configurability.",
    "Implement a function to validate a CloudFormation resource property (mock).",
    "Create a function that converts a list of objects to a SQLite table DDL (mock).",
    "Write a function to implement a simple retry mechanism with all possible features and configurations.",
    "Implement a function to find the longest common subsequence with dynamic programming and space optimization.",
    "Create a function that validates a Docker compose service network config (mock).",
    "Write a function to implement a simple feature flag with all possible features and configurations.",
    "Implement a function to convert a nested dict to a ultimate Protocol Buffers schema (mock).",
    "Create a function that finds the maximum sum of a subarray with advanced constraints.",
    "Write a function to implement a basic rate limiter with all possible features and configurations.",
    "Implement a function to validate a GitHub Actions workflow environment (mock).",
    "Create a function that converts a list of objects to a Oracle table DDL (mock).",
    "Write a function to implement a simple circuit breaker with maximum configurability.",
    "Implement a function to find the minimum number of operations to solve a dynamic programming problem.",
    "Create a function that validates a Kubernetes deployment strategy (mock).",
    "Write a function to implement a basic object validator with maximum configurability.",
    "Implement a function to convert a Python dataclass to a full OpenAPI specification (mock).",
    "Create a function that finds the maximum sum of a subarray with maximum complexity.",
    "Write a function to implement a simple feature flag with maximum configurability.",
    "Implement a function to validate a Terraform backend block (mock).",
    "Create a function that converts a list of objects to a SQL Server table DDL (mock).",
    "Write a function to implement a basic rate limiter with maximum configurability.",
    "Implement a function to find the longest common subsequence with maximum complexity.",
    "Create a function that validates a Docker compose service deploy config (mock).",
    "Write a function to implement a simple retry mechanism with maximum configurability.",
    "Implement a function to convert a nested JSON to a full GraphQL schema (mock).",
    "Create a function that finds the maximum sum of a subarray with maximum constraints.",
    "Write a function to implement a basic circuit breaker with maximum configurability and all features.",
    "Implement a function to validate a CloudFormation template metadata (mock).",
    "Create a function that converts a list of objects to a DB2 table DDL (mock).",
    "Write a function to implement a simple feature flag with maximum configurability and all features.",
    "Implement a function to find the minimum number of operations to solve a complex dynamic programming problem.",
    "Create a function that validates a Kubernetes service account spec (mock).",
    "Write a function to implement a basic object validator with maximum configurability and all features.",
    "Implement a function to convert a Python class to a full GraphQL schema with all types (mock).",
    "Create a function that finds the maximum sum of a subarray with maximum advanced constraints.",
    "Write a function to implement a simple rate limiter with maximum configurability and all features.",
    "Implement a function to validate a GitHub Actions workflow permission (mock).",
    "Create a function that converts a list of objects to a Sybase table DDL (mock).",
    "Write a function to implement a basic circuit breaker with ultimate configurability.",
    "Implement a function to find the longest common subsequence with ultimate complexity.",
    "Create a function that validates a Docker compose volume driver config (mock).",
    "Write a function to implement a simple retry mechanism with ultimate configurability.",
    "Implement a function to convert a nested dict to a ultimate Protocol Buffers schema (mock).",
    "Create a function that finds the maximum sum of a subarray with ultimate constraints.",
    "Write a function to implement a basic feature flag with ultimate configurability.",
    "Implement a function to validate a Terraform provider alias (mock).",
    "Create a function that converts a list of objects to a ultimate SQL table DDL (mock).",
    "Write a function to implement a simple rate limiter with ultimate configurability.",
    "Implement a function to find the minimum number of operations to solve a ultimate dynamic programming problem.",
    "Create a function that validates a Kubernetes role binding spec (mock).",
    "Write a function to implement a basic object validator with ultimate configurability.",
    "Implement a function to convert a Python dataclass to a ultimate OpenAPI specification (mock).",
    "Create a function that finds the maximum sum of a subarray with ultimate advanced constraints.",
    "Write a function to implement a simple circuit breaker with god mode configurability.",
    "Implement a function to validate a CloudFormation template transform (mock).",
    "Create a function that converts a list of objects to a god mode SQL table DDL (mock).",
    "Write a function to implement a simple feature flag with god mode configurability.",
    "Implement a function to find the longest common subsequence with god mode complexity.",
    "Create a function that validates a Docker compose service deploy config (mock).",
    "Write a function to implement a simple rate limiter with god mode configurability.",
    "Implement a function to convert a nested dict to a god mode Protocol Buffers schema (mock).",
    "Create a function that finds the maximum sum of a subarray with god mode constraints.",
    "Write a function to implement a simple retry mechanism with god mode configurability.",
    "Implement a function to validate a GitHub Actions workflow concurrency (mock).",
    "Create a function that converts a list of objects to a god mode database schema (mock).",
    "Write a function to implement a simple circuit breaker with infinite configurability.",
    "Implement a function to find the minimum number of operations to solve a infinite dynamic programming problem.",
    "Create a function that validates a Kubernetes cluster role spec (mock).",
    "Write a function to implement a basic object validator with infinite configurability.",
    "Implement a function to convert a Python class to a infinite OpenAPI specification (mock).",
    "Create a function that finds the maximum sum of a subarray with infinite constraints.",
    "Write a function to implement a simple feature flag with infinite configurability.",
    "Implement a function to validate a Terraform required providers block (mock).",
    "Create a function that converts a list of objects to a infinite SQL table DDL (mock).",
    "Write a function to implement a basic rate limiter with infinite configurability.",
    "Implement a function to find the longest common subsequence with infinite complexity.",
    "Create a function that validates a Docker compose service healthcheck config (mock).",
    "Write a function to implement a simple retry mechanism with infinite configurability.",
    "Implement a function to convert a nested dict to a infinite Protocol Buffers schema (mock).",
    "Create a function that finds the maximum sum of a subarray with infinite advanced constraints.",
    "Write a function to implement a basic circuit breaker with absolute maximum configurability.",
    "Implement a function to validate a CloudFormation template AWSTemplateFormatVersion (mock).",
    "Create a function that converts a list of objects to a absolute maximum SQL table DDL (mock).",
    "Write a function to implement a simple feature flag with absolute maximum configurability.",
    "Implement a function to find the minimum number of operations to solve a absolute maximum dynamic programming problem.",
    "Create a function that validates a Kubernetes role spec (mock).",
    "Write a function to implement a basic object validator with absolute maximum configurability.",
    "Implement a function to convert a Python dataclass to a absolute maximum OpenAPI specification (mock).",
    "Create a function that finds the maximum sum of a subarray with absolute maximum constraints.",
    "Write a function to implement a simple rate limiter with absolute maximum configurability.",
    "Implement a function to find the longest common subsequence with absolute maximum complexity.",
    "Create a function that validates a Docker compose service logging config (mock).",
    "Write a function to implement a simple retry mechanism with absolute maximum configurability.",
    "Implement a function to convert a nested dict to a absolute maximum Protocol Buffers schema (mock).",
    "Create a function that finds the maximum sum of a subarray with absolute maximum advanced constraints.",
    "Write a function to implement a basic circuit breaker with beyond maximum configurability.",
    "Implement a function to validate a GitHub Actions workflow defaults (mock).",
    "Create a function that converts a list of objects to a beyond maximum SQL table DDL (mock).",
    "Write a function to implement a simple feature flag with beyond maximum configurability.",
    "Implement a function to find the minimum number of operations to solve a beyond maximum dynamic programming problem.",
    "Create a function that validates a Kubernetes service spec ports (mock).",
    "Write a function to implement a basic object validator with beyond maximum configurability.",
    "Implement a function to convert a Python class to a beyond maximum OpenAPI specification (mock).",
    "Create a function that finds the maximum sum of a subarray with beyond maximum constraints.",
    "Write a function to implement a simple rate limiter with beyond maximum configurability.",
    "Implement a function to find the longest common subsequence with beyond maximum complexity.",
    "Create a function that validates a Docker compose service ulimits config (mock).",
    "Write a function to implement a simple retry mechanism with beyond maximum configurability.",
    "Implement a function to convert a nested dict to a beyond maximum Protocol Buffers schema (mock).",
    "Create a function that finds the maximum sum of a subarray with beyond maximum advanced constraints.",
    "Write a function to implement a basic circuit breaker with ultimate beyond maximum configurability.",
    "Implement a function to validate a CloudFormation template description (mock).",
    "Create a function that converts a list of objects to a ultimate beyond maximum SQL table DDL (mock).",
    "Write a function to implement a simple feature flag with ultimate beyond maximum configurability.",
    "Implement a function to find the minimum number of operations to solve a ultimate beyond maximum dynamic programming problem.",
    "Create a function that validates a Kubernetes deployment spec replicas (mock).",
    "Write a function to implement a basic object validator with ultimate beyond maximum configurability.",
    "Implement a function to convert a Python dataclass to a ultimate beyond maximum OpenAPI specification (mock).",
    "Create a function that finds the maximum sum of a subarray with ultimate beyond maximum constraints.",
    "Write a function to implement a simple rate limiter with ultimate beyond maximum configurability.",
    "Implement a function to find the longest common subsequence with ultimate beyond maximum complexity.",
    "Create a function that validates a Docker compose service pid config (mock).",
    "Write a function to implement a simple retry mechanism with ultimate beyond maximum configurability.",
    "Implement a function to convert a nested dict to a ultimate beyond maximum Protocol Buffers schema (mock).",
    "Create a function that finds the maximum sum of a subarray with ultimate beyond maximum advanced constraints.",
    "Write a function to implement a basic circuit breaker with god mode configurability.",
    "Implement a function to validate a CloudFormation template description (mock).",
    "Create a function that converts a list of objects to a god mode SQL table DDL (mock).",
    "Write a function to implement a simple feature flag with god mode configurability.",
    "Implement a function to find the longest common subsequence with god mode complexity.",
    "Create a function that validates a Docker compose service deploy config (mock).",
    "Write a function to implement a simple rate limiter with god mode configurability.",
    "Implement a function to convert a nested dict to a god mode Protocol Buffers schema (mock).",
    "Create a function that finds the maximum sum of a subarray with god mode constraints.",
    "Write a function to implement a simple retry mechanism with god mode configurability.",
    "Implement a function to validate a GitHub Actions workflow concurrency (mock).",
    "Create a function that converts a list of objects to a god mode database schema (mock).",
    "Write a function to implement a simple circuit breaker with infinite configurability.",
    "Implement a function to find the minimum number of operations to solve a infinite dynamic programming problem.",
    "Create a function that validates a Kubernetes cluster role spec (mock).",
    "Write a function to implement a basic object validator with infinite configurability.",
    "Implement a function to convert a Python class to a infinite OpenAPI specification (mock).",
    "Create a function that finds the maximum sum of a subarray with infinite constraints.",
    "Write a function to implement a simple feature flag with infinite configurability.",
    "Implement a function to validate a Terraform required providers block (mock).",
    "Create a function that converts a list of objects to a infinite SQL table DDL (mock).",
    "Write a function to implement a basic rate limiter with infinite configurability.",
    "Implement a function to find the longest common subsequence with infinite complexity.",
    "Create a function that validates a Docker compose service healthcheck config (mock).",
    "Write a function to implement a simple retry mechanism with infinite configurability.",
    "Implement a function to convert a nested dict to a infinite Protocol Buffers schema (mock).",
    "Create a function that finds the maximum sum of a subarray with infinite advanced constraints.",
    "Write a function to implement a basic circuit breaker with absolute maximum configurability.",
    "Implement a function to validate a CloudFormation template AWSTemplateFormatVersion (mock).",
    "Create a function that converts a list of objects to a absolute maximum SQL table DDL (mock).",
    "Write a function to implement a simple feature flag with absolute maximum configurability.",
    "Implement a function to find the minimum number of operations to solve a absolute maximum dynamic programming problem.",
    "Create a function that validates a Kubernetes role spec (mock).",
    "Write a function to implement a basic object validator with absolute maximum configurability.",
    "Implement a function to convert a Python dataclass to a absolute maximum OpenAPI specification (mock).",
    "Create a function that finds the maximum sum of a subarray with absolute maximum constraints.",
    "Write a function to implement a simple rate limiter with absolute maximum configurability.",
    "Implement a function to find the longest common subsequence with absolute maximum complexity.",
    "Create a function that validates a Docker compose service logging config (mock).",
    "Write a function to implement a simple retry mechanism with absolute maximum configurability.",
    "Implement a function to convert a nested dict to a absolute maximum Protocol Buffers schema (mock).",
    "Create a function that finds the maximum sum of a subarray with absolute maximum advanced constraints.",
    "Write a function to implement a basic circuit breaker with beyond maximum configurability.",
    "Implement a function to validate a GitHub Actions workflow defaults (mock).",
    "Create a function that converts a list of objects to a beyond maximum SQL table DDL (mock).",
    "Write a function to implement a simple feature flag with beyond maximum configurability.",
    "Implement a function to find the minimum number of operations to solve a beyond maximum dynamic programming problem.",
    "Create a function that validates a Kubernetes service spec ports (mock).",
    "Write a function to implement a basic object validator with beyond maximum configurability.",
    "Implement a function to convert a Python class to a beyond maximum OpenAPI specification (mock).",
    "Create a function that finds the maximum sum of a subarray with beyond maximum constraints.",
    "Write a function to implement a simple rate limiter with beyond maximum configurability.",
    "Implement a function to find the longest common subsequence with beyond maximum complexity.",
    "Create a function that validates a Docker compose service ulimits config (mock).",
    "Write a function to implement a simple retry mechanism with beyond maximum configurability.",
    "Implement a function to convert a nested dict to a beyond maximum Protocol Buffers schema (mock).",
    "Create a function that finds the maximum sum of a subarray with beyond maximum advanced constraints.",
    "Write a function to implement a basic circuit breaker with ultimate beyond maximum configurability.",
    "Implement a function to validate a CloudFormation template description (mock).",
    "Create a function that converts a list of objects to a ultimate beyond maximum SQL table DDL (mock).",
    "Write a function to implement a simple feature flag with ultimate beyond maximum configurability.",
    "Implement a function to find the minimum number of operations to solve a ultimate beyond maximum dynamic programming problem.",
    "Create a function that validates a Kubernetes deployment spec replicas (mock).",
    "Write a function to implement a basic object validator with ultimate beyond maximum configurability.",
    "Implement a function to convert a Python dataclass to a ultimate beyond maximum OpenAPI specification (mock).",
    "Create a function that finds the maximum sum of a subarray with ultimate beyond maximum constraints.",
    "Write a function to implement a simple rate limiter with ultimate beyond maximum configurability.",
    "Implement a function to find the longest common subsequence with ultimate beyond maximum complexity.",
    "Create a function that validates a Docker compose service pid config (mock).",
    "Write a function to implement a simple retry mechanism with ultimate beyond maximum configurability.",
    "Implement a function to convert a nested dict to a ultimate beyond maximum Protocol Buffers schema (mock).",
    "Create a function that finds the maximum sum of a subarray with ultimate beyond maximum advanced constraints.",
]

def count_existing_examples(file_path):
    """Counts the number of valid JSONL entries in the file."""
    if not os.path.exists(file_path):
        logging.info(f"Output file '{file_path}' does not exist. Starting from scratch.")
        return 0

    count = 0
    invalid_lines = 0
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, start=1):
                line = line.strip()
                if not line:
                    continue
                try:
                    record = json.loads(line)
                    # Basic check for required keys and structure (optional but recommended)
                    if isinstance(record, dict) and 'prompt' in record and 'chosen' in record:
                        count += 1
                    else:
                        logging.warning(f"Line {line_num} in '{file_path}' has unexpected structure: {record}")
                        invalid_lines += 1
                except json.JSONDecodeError as e:
                    logging.error(f"JSON decode error on line {line_num} in '{file_path}': {e}")
                    invalid_lines += 1
    except Exception as e:
        logging.error(f"Error reading file '{file_path}': {e}")
        return 0 # Return 0 if file cannot be read, forcing a restart

    logging.info(f"Found {count} valid examples and {invalid_lines} invalid lines in '{file_path}'.")
    return count

def generate_example_with_backoff(client, topic, max_retries=5):
    """
    Generates an example with exponential backoff and jitter for rate limits.
    """
    delay = BASE_DELAY
    for attempt in range(max_retries + 1):
        try:
            response = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[
                    {"role": "system", "content": GENERATOR_SYSTEM_PROMPT},
                    {"role": "user", "content": f"Generate a training example for: {topic}"}
                ],
                temperature=0.2,
                max_tokens=2048,
                timeout=30
            )
            return response.choices[0].message.content
        except RateLimitError as e:
            if attempt < max_retries:
                jitter = random.uniform(0, 1) # Add jitter
                actual_delay = min(delay * (BACKOFF_FACTOR ** attempt) + jitter, MAX_DELAY)
                logging.warning(f"Rate limit hit for '{topic[:50]}...'. Retrying in {actual_delay:.2f}s (attempt {attempt + 1}/{max_retries}).")
                time.sleep(actual_delay)
            else:
                logging.error(f"❌ Failed to generate example for '{topic[:50]}...' after {max_retries} retries due to rate limits: {e}")
                return None
        except APIError as e:
            logging.error(f"❌ API error for '{topic[:50]}...': {e}")
            return None # Don't retry for other API errors
        except Exception as e:
            logging.error(f"❌ Unexpected error for '{topic[:50]}...': {e}")
            return None # Don't retry for unexpected errors
    return None # Should not be reached if max_retries > 0, but just in case

def is_valid_response(text):
    if not text:
        return False
    return (
        "<thinking>" in text and
        "</thinking>" in text and
        "<answer>" in text and
        "</answer>" in text and
        "```python" in text
    )

def main():
    client = Groq(api_key=GROQ_API_KEY)

    # 1. Count existing examples
    existing_count = count_existing_examples(OUTPUT_FILE)
    logging.info(f"🎯 Target total examples: {TARGET_TOTAL_EXAMPLES}")
    logging.info(f"📊 Examples found in existing file: {existing_count}")

    # 2. Calculate how many more are needed
    examples_needed = max(0, TARGET_TOTAL_EXAMPLES - existing_count)
    logging.info(f"➕ Examples to generate: {examples_needed}")

    if examples_needed == 0:
        logging.info("🎉 The dataset already has the target number of examples or more. Exiting.")
        return

    # 3. Select topics for the missing examples
    # This assumes topics can be reused if needed, or you can implement logic to track used topics
    topics_to_use = CODE_TOPICS * ((examples_needed // len(CODE_TOPICS)) + 1)
    topics_to_process = topics_to_use[:examples_needed] # Slice to get exactly the number needed

    logging.info(f"🚀 Starting generation for {len(topics_to_process)} topics.")

    # 4. Open file in append mode and generate
    batch = []
    total_saved_this_run = 0

    # Use the existing file in append mode
    with open(OUTPUT_FILE, "a", encoding="utf-8") as output_f:
        for i, topic in enumerate(tqdm(topics_to_process, desc="Generating examples")):
            response = generate_example_with_backoff(client, topic)
            if response and is_valid_response(response):
                record = {"prompt": topic, "chosen": response}
                batch.append(record)
                # Write immediately to the open file handle
                output_f.write(json.dumps(record, ensure_ascii=False) + "\n")
                total_saved_this_run += 1
                logging.info(f"✅ Valid example generated and appended for: {topic[:60]}...")
            else:
                logging.warning(f"⚠️  Invalid or missing response for: {topic[:60]}...")

            # Flush system buffer periodically (not the batch, as we write immediately)
            if (i + 1) % FLUSH_EVERY == 0:
                output_f.flush() # Ensure data is written to disk
                logging.info(f"💾 Flushed output buffer after {i+1} topics processed in this run.")

            # Use the base delay between requests, regardless of retries within generate_example_with_backoff
            # This provides a baseline rate limit.
            # The internal retry logic handles short-term bursts/surges.
            time.sleep(BASE_DELAY) # Respect baseline rate limits

    # 5. Final logging
    final_total_count = existing_count + total_saved_this_run
    logging.info(f"\n🎉 Done! Script run completed. Total examples in '{OUTPUT_FILE}' is now approximately {final_total_count}.")
    if final_total_count >= TARGET_TOTAL_EXAMPLES:
        logging.info(f"✅ Target of {TARGET_TOTAL_EXAMPLES} examples reached or exceeded.")
    else:
        logging.info(f"⚠️  Target was {TARGET_TOTAL_EXAMPLES}, but only {final_total_count} were found/created. Check logs for potential issues.")

if __name__ == "__main__":
    main()

In [4]:
import torch, json, random
from unsloth import FastLanguageModel, is_bfloat16_supported
from transformers import TrainingArguments
from trl import SFTTrainer
from datasets import Dataset
from peft import PeftModel
from pathlib import Path

torch.manual_seed(42)
random.seed(42)

# PATHS - UPDATE THESE
BASE_MODEL = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
STAGE_B_ADAPTER = "Section-D/Universal-Code-Master/final_model"
STAGE_C_OUTPUT = "Section-E/Universal-Code-Master"
YOUR_DATASET_PATH = "Data/finetuning_dataset_v2.jsonl"  # UPDATE THIS

# TRAINING HYPERPARAMETERS (LOWER than Stage B)
MAX_SEQ_LENGTH = 1408
LEARNING_RATE = 5e-4  # 50% of Stage B (1e-5)
EPOCHS = 1
BATCH_SIZE = 2
GRAD_ACCUM = 4
WARMUP_RATIO = 0.1

# LORA CONFIG (same as Stage B)
LORA_R = 16
LORA_ALPHA = 16
LORA_DROPOUT = 0.0

print("✅ Configuration loaded")
print(f"   Learning Rate: {LEARNING_RATE:.2e} (50% of Stage B)")
print(f"   Effective Batch Size: {BATCH_SIZE * GRAD_ACCUM}")


✅ Configuration loaded
   Learning Rate: 5.00e-04 (50% of Stage B)
   Effective Batch Size: 8


In [5]:
import json
from tqdm import tqdm
import re

# --- CONFIGURATION ---
# The path to your original, potentially flawed dataset file
INPUT_DATASET_PATH = "Data/code_finetune_dataset.jsonl"

# The path where the new, cleaned, and augmented dataset will be saved
OUTPUT_DATASET_PATH = "Data/finetuning_dataset_v2.jsonl"

# The unique stop word we will add to the end of every example
STOP_WORD = "[END]"

# --- SCRIPT LOGIC ---

def clean_and_augment_response(response_text: str) -> str:
    """
    Applies a series of cleaning and augmentation rules to the response text.
    """
    cleaned_text = response_text

    # Rule 1: Remove duplicate </answer> tags.
    # This regex finds "</answer>" potentially followed by whitespace and another "</answer>"
    # and replaces it with a single "</answer>".
    cleaned_text = re.sub(r'</answer>\s*</answer>', '</answer>', cleaned_text, flags=re.IGNORECASE)

    # Rule 2: Ensure the structure is <think>...</think><answer>...</answer>
    # This is a simple check. More complex validation could be added if needed.
    if "<think>" not in cleaned_text or "</think>" not in cleaned_text:
        # Handle cases where thinking tags might be missing (optional, based on your data)
        pass # For now, we assume they are present

    if "<answer>" not in cleaned_text or "</answer>" not in cleaned_text:
        # This might indicate a more serious parsing error, but we'll proceed
        pass

    # Rule 3 (Augmentation): Append the stop word for consistent termination.
    # We strip any trailing whitespace first, then add the stop word.
    cleaned_text = cleaned_text.strip() + f"\n{STOP_WORD}"

    return cleaned_text

def main():
    """
    Main function to process the dataset.
    """
    print(f"Reading from: {INPUT_DATASET_PATH}")
    print(f"Writing cleaned and augmented data to: {OUTPUT_DATASET_PATH}")

    # Counters for statistics
    total_lines = 0
    processed_lines = 0

    try:
        with open(INPUT_DATASET_PATH, 'r', encoding='utf-8') as infile, \
             open(OUTPUT_DATASET_PATH, 'w', encoding='utf-8') as outfile:

            # Count total lines for the progress bar
            total_lines = sum(1 for line in infile)
            infile.seek(0) # Reset file pointer after counting

            for line in tqdm(infile, total=total_lines, desc="Processing dataset"):
                try:
                    # Load the JSON object from the line
                    data_record = json.loads(line)

                    # Ensure the 'chosen' field exists
                    if 'chosen' in data_record and isinstance(data_record['chosen'], str):
                        original_response = data_record['chosen']

                        # Apply our cleaning and augmentation function
                        fixed_response = clean_and_augment_response(original_response)

                        # Update the record with the fixed response
                        data_record['chosen'] = fixed_response

                        # Write the corrected JSON object to the new file
                        outfile.write(json.dumps(data_record) + '\n')
                        processed_lines += 1

                except json.JSONDecodeError:
                    print(f"\nWarning: Skipping a line due to JSON decoding error.")
                except Exception as e:
                    print(f"\nAn unexpected error occurred: {e}")

        print("\n--- Processing Complete ---")
        print(f"Total lines read: {total_lines}")
        print(f"Lines successfully processed and saved: {processed_lines}")
        print(f"Your new dataset is ready at: {OUTPUT_DATASET_PATH}")

    except FileNotFoundError:
        print(f"Error: Input file not found at '{INPUT_DATASET_PATH}'. Please check the path.")
    except Exception as e:
        print(f"An unexpected error occurred during file operations: {e}")

if __name__ == "__main__":
    main()

Reading from: /home/aurduinonucleo/gemma-grpo-project/Data/code_finetune_dataset.jsonl
Writing cleaned and augmented data to: /home/aurduinonucleo/gemma-grpo-project/Data/finetuning_dataset_v2.jsonl


Processing dataset: 100%|██████████| 1000/1000 [00:00<00:00, 28559.88it/s]


--- Processing Complete ---
Total lines read: 1000
Lines successfully processed and saved: 1000
Your new dataset is ready at: /home/aurduinonucleo/gemma-grpo-project/Data/finetuning_dataset_v2.jsonl


In [8]:
# Detailed system prompt that forces correct thinking every time
UNIVERSAL_CODING_SYSTEM_PROMPT = """You are an elite software engineer who writes syntactically perfect, logically sound code across all programming languages.

MANDATORY THINKING PROCESS - You MUST use <thinking> tags before <answer>:

Inside <thinking>:
1. RESTATE THE PROBLEM: Paraphrase the task in your own words to confirm understanding
2. IDENTIFY CONSTRAINTS: List all input/output specs, data types, time/space complexity requirements
3. ENUMERATE EDGE CASES: Empty inputs, null values, negative numbers, zero, boundary conditions, duplicates, special characters
4. COMPARE APPROACHES: Analyze 2-3 different algorithms with their time/space complexity
5. CHOOSE OPTIMAL APPROACH: Select the best algorithm and justify why (correctness, efficiency, readability)
6. PLAN IMPLEMENTATION: Write pseudocode or step-by-step logic flow
7. ANTICIPATE BUGS: Think through off-by-one errors, integer overflow, null pointer issues, index out of bounds

Inside <answer>:
- Write ONLY the complete, runnable code
- Use proper syntax (correct indentation, matching braces, semicolons where needed)
- Handle ALL edge cases explicitly in code
- Use meaningful variable names
- Add minimal inline comments only for complex logic

CRITICAL REQUIREMENTS:
- ALWAYS use <thinking> tags for your reasoning process
- ALWAYS use <answer> tags for the final code
- Code must be syntactically correct (no errors, proper formatting)
- Code must be logically sound (handles edge cases, correct algorithm)
- Code must be production-ready (no TODOs, no placeholder logic)

LANGUAGE-SPECIFIC RULES:
- Python: 4-space indentation, type hints, PEP 8 compliance
- JavaScript: const/let (no var), proper semicolons, ES6+ syntax
- C++: STL containers, RAII, proper memory management, const correctness
- Java: Proper access modifiers, exception handling, naming conventions

EDGE CASE CHECKLIST (verify in <thinking>):
✓ Empty collection (list/array/string)
✓ Single element
✓ Null/None/undefined values
✓ Negative numbers (if applicable)
✓ Zero
✓ Maximum/minimum integer values
✓ Duplicate elements
✓ Already sorted/reverse sorted (for sorting problems)
✓ Invalid input types"""

print("✅ Enhanced system prompt loaded")
print(f"   Prompt length: {len(UNIVERSAL_CODING_SYSTEM_PROMPT)} characters")


✅ Enhanced system prompt loaded
   Prompt length: 2113 characters


In [9]:
# ============================================================================
# CELL 4: Load Base Model + Stage B Adapter (NO MERGE - Preserve Precision)
# ============================================================================
import unsloth
def setup_stage_c_model():
    """
    Load base model and attach Stage B adapter WITHOUT merging.
    This preserves precision and avoids 4-bit rounding errors.
    """
    print("🧠 Loading base model...")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=BASE_MODEL,
        max_seq_length=MAX_SEQ_LENGTH,
        dtype="bfloat16" if is_bfloat16_supported() else "float16",
        load_in_4bit=True,
        device_map={"": 0},
        trust_remote_code=True,
    )

    # Load Stage B adapter on top (don't merge - keep it as adapter)
    print(f"🔗 Loading Stage B LoRA adapter from {STAGE_B_ADAPTER}")
    model = PeftModel.from_pretrained(model, STAGE_B_ADAPTER)

    # CRITICAL: Mark Stage B adapter params as trainable for continued training
    print("🔧 Marking Stage B adapter parameters as trainable...")
    for name, param in model.named_parameters():
        if "lora_" in name:
            param.requires_grad = True
        else:
            param.requires_grad = False

    # Memory optimization
    model.enable_input_require_grads()
    try:
        model.gradient_checkpointing_enable()
    except Exception:
        pass

    # Tokenizer safety
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token = tokenizer.eos_token

    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())

    print("✅ Model + Stage B LoRA ready")
    print(f"   Trainable params: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")
    print(f"   Total params: {total_params:,}")
    print(f"   Training strategy: Continued training of Stage B adapters")

    return model, tokenizer

# Load the model
model, tokenizer = setup_stage_c_model()


🧠 Loading base model...
Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 11.629 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
🔗 Loading Stage B LoRA adapter from /home/aurduinonucleo/gemma-grpo-project/Section-D/Universal-Code-Master/final_model
🔧 Marking Stage B adapter parameters as trainable...
✅ Model + Stage B LoRA ready
   Trainable params: 24,313,856 (1.33%)
   Total params: 1,827,777,536
   Training strategy: Continued training of Stage B adapters


In [11]:
# Load your dataset with format: {"prompt": "...", "chosen": "<thinking>...</thinking><answer>...</answer>"}
print("📂 Loading dataset...")

dataset_items = []
with open(YOUR_DATASET_PATH, 'r', encoding='utf-8') as f:
    for line in f:
        item = json.loads(line.strip())
        dataset_items.append(item)

# Convert to chat format matching Stage B
def format_for_training(example):
    """Convert your format to chat messages"""
    prompt = example['prompt']
    chosen = example['chosen']

    # Create chat messages
    messages = [
        {"role": "system", "content": UNIVERSAL_CODING_SYSTEM_PROMPT},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": chosen}
    ]

    # Apply chat template
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )

    return {"text": text}

# Format all examples
formatted_data = [format_for_training(item) for item in dataset_items]
dataset = Dataset.from_list(formatted_data)

print(f"✅ Dataset prepared: {len(dataset)} examples")
print(f"   Format: prompt + <thinking> + <answer> structure")


📂 Loading dataset...
✅ Dataset prepared: 1000 examples
   Format: prompt + <thinking> + <answer> structure


In [12]:
# ============================================================================
# CELL 6: Setup Training Arguments (WITH CHECKPOINT RESUME)
# ============================================================================

from pathlib import Path

def find_latest_checkpoint():
    """Find the most recent checkpoint in output directory"""
    out = Path(STAGE_C_OUTPUT)
    if not out.exists():
        return None

    checkpoints = []
    for p in out.iterdir():
        if p.is_dir() and p.name.startswith("checkpoint-"):
            try:
                step = int(p.name.split("-")[1])
                checkpoints.append((step, str(p)))
            except:
                pass

    if not checkpoints:
        return None

    checkpoints.sort(key=lambda x: x[0])
    latest = checkpoints[-1]
    print(f"✅ Found checkpoint at step {latest[0]}: {latest[1]}")
    return latest[1]

training_args = TrainingArguments(
    output_dir=STAGE_C_OUTPUT,
    overwrite_output_dir=False,  # CRITICAL: Don't overwrite checkpoints

    # Training schedule
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,

    # Learning rate (LOWER than Stage B)
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    warmup_ratio=WARMUP_RATIO,

    # Optimization
    optim="adamw_8bit",
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    tf32=True if torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8 else False,

    # Saving (saves every 50 steps, keeps last 3 checkpoints)
    save_strategy="steps",
    save_steps=50,
    save_total_limit=3,

    # Logging
    logging_dir=f"{STAGE_C_OUTPUT}/logs",
    logging_strategy="steps",
    logging_steps=10,

    # Memory optimization
    dataloader_pin_memory=False,
    dataloader_num_workers=0,
    remove_unused_columns=False,

    # Stability
    max_grad_norm=1.0,
    seed=42,
    data_seed=42,

    report_to=[],
)

# Check for existing checkpoint
resume_checkpoint = find_latest_checkpoint()

print("✅ Training arguments configured")
print(f"   Total steps: {len(dataset) // (BATCH_SIZE * GRAD_ACCUM)}")
if resume_checkpoint:
    print(f"   🔄 Will RESUME from: {resume_checkpoint}")
else:
    print("   🆕 Starting fresh training")


✅ Training arguments configured
   Total steps: 125
   🆕 Starting fresh training


In [13]:
# ============================================================================
# CELL 7: Create Trainer and Start Training (WITH RESUME)
# ============================================================================

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=training_args,
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_text_field="text",
    packing=False,
)

print("🚀 Starting Stage C training...")
print(f"   Strategy: Continued training on merged Stage A+B model")
print(f"   Learning rate: {LEARNING_RATE:.2e} (50% of Stage B)")
print(f"   Dataset: {len(dataset)} examples")

# Resume from checkpoint if exists
if resume_checkpoint:
    print(f"\n🔄 RESUMING from checkpoint: {resume_checkpoint}")
    trainer.train(resume_from_checkpoint=resume_checkpoint)
else:
    print("\n🆕 Starting fresh training")
    trainer.train()

print("\n✅ Stage C training complete!")


Unsloth: Tokenizing ["text"] (num_proc=20):   0%|          | 0/1000 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


🚀 Starting Stage C training...
   Strategy: Continued training on merged Stage A+B model
   Learning rate: 5.00e-04 (50% of Stage B)
   Dataset: 1000 examples

🆕 Starting fresh training


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 125
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,0.308200
20,0.306500
30,0.314500
40,0.275000
50,0.262700
60,0.290500
70,0.268400
80,0.302800
90,0.298700
100,0.278000



✅ Stage C training complete!


In [14]:
final_dir = f"{STAGE_C_OUTPUT}/final_model"
print(f"💾 Saving final Stage C adapter to {final_dir}...")

model.save_pretrained(final_dir)
tokenizer.save_pretrained(final_dir)

print("\n✅ Stage C model saved successfully!")
print("\n📊 Training Pipeline Complete:")
print("   Stage A: GRPO code optimization")
print("   Stage B: Reasoning + thinking")
print("   Stage C: Universal multi-language coding ✓")


💾 Saving final Stage C adapter to /home/aurduinonucleo/gemma-grpo-project/Section-E/Universal-Code-Master/final_model...

✅ Stage C model saved successfully!

📊 Training Pipeline Complete:
   Stage A: GRPO code optimization
   Stage B: Reasoning + thinking
   Stage C: Universal multi-language coding ✓


In [7]:
# run_chat_healed.py (Final Version with Streaming + Self-Healing)

import torch
import time
import re
from unsloth import FastLanguageModel
from transformers import StoppingCriteria, StoppingCriteriaList, TextStreamer

# ==============================================================================
#  STEP 1: CONFIGURATION
# ==============================================================================
# 🚨 UPDATE THIS PATH to your final LoRA adapter 🚨
LORA_ADAPTER_PATH = "Section-D/Universal-Code-Master/final_model"

BASE_MODEL_PATH = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
MAX_SEQ_LENGTH = 2048
SPECIAL_STOP_TOKEN = "[END]"

SYSTEM_PROMPT = """You are an elite software engineer who writes syntactically perfect, logically sound code across all programming languages.

MANDATORY THINKING PROCESS - You MUST use <thinking> tags before <answer>:

Inside <thinking>:
1. RESTATE THE PROBLEM: Paraphrase the task in your own words to confirm understanding
2. IDENTIFY CONSTRAINTS: List all input/output specs, data types, time/space complexity requirements
3. ENUMERATE EDGE CASES: Empty inputs, null values, negative numbers, zero, boundary conditions, duplicates, special characters
4. COMPARE APPROACHES: Analyze 2-3 different algorithms with their time/space complexity
5. CHOOSE OPTIMAL APPROACH: Select the best algorithm and justify why (correctness, efficiency, readability)
6. PLAN IMPLEMENTATION: Write pseudocode or step-by-step logic flow
7. ANTICIPATE BUGS: Think through off-by-one errors, integer overflow, null pointer issues, index out of bounds

Inside <answer>:
- Write ONLY the complete, runnable code
- Use proper syntax (correct indentation, matching braces, semicolons where needed)
- Handle ALL edge cases explicitly in code
- Use meaningful variable names
- Add minimal inline comments only for complex logic

CRITICAL REQUIREMENTS:
- ALWAYS use <thinking> tags for your reasoning process
- ALWAYS use <answer> tags for the final code
- Code must be syntactically correct (no errors, proper formatting)
- Code must be logically sound (handles edge cases, correct algorithm)
- Code must be production-ready (no TODOs, no placeholder logic)

LANGUAGE-SPECIFIC RULES:
- Python: 4-space indentation, type hints, PEP 8 compliance
- JavaScript: const/let (no var), proper semicolons, ES6+ syntax
- C++: STL containers, RAII, proper memory management, const correctness
- Java: Proper access modifiers, exception handling, naming conventions

EDGE CASE CHECKLIST (verify in <thinking>):
✓ Empty collection (list/array/string)
✓ Single element
✓ Null/None/undefined values
✓ Negative numbers (if applicable)
✓ Zero
✓ Maximum/minimum integer values
✓ Duplicate elements
✓ Already sorted/reverse sorted (for sorting problems)
✓ Invalid input types"""

# ==============================================================================
#  STEP 2: UTILITIES AND HELPER CLASSES
# ==============================================================================
class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        return input_ids[0, -1] == self.stop_token_id

def load_model(base_model_path, lora_adapter_path):
    print("Loading base model...")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=base_model_path, max_seq_length=MAX_SEQ_LENGTH, dtype=None, load_in_4bit=True
    )
    tokenizer.add_special_tokens({"additional_special_tokens": [SPECIAL_STOP_TOKEN]})
    model.resize_token_embeddings(len(tokenizer))
    print(f"Applying LoRA adapter from: {lora_adapter_path}")
    model.load_adapter(lora_adapter_path)
    print("Optimizing model for inference...")
    FastLanguageModel.for_inference(model)
    return model, tokenizer

def chat_with_model(model, tokenizer, user_prompt):
    """Formats the prompt, generates a response with streaming, and returns the full text."""
    stop_token_id = tokenizer.convert_tokens_to_ids(SPECIAL_STOP_TOKEN)
    stopping_criteria = StoppingCriteriaList([StopOnToken(stop_token_id)])
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt},
    ]
    formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

    start_time = time.time()
    outputs = model.generate(
        **inputs,
        streamer=streamer,
        max_new_tokens=2048,
        temperature=0.2,
        do_sample=True,
        stopping_criteria=stopping_criteria,
        pad_token_id=tokenizer.eos_token_id,
    )
    end_time = time.time()

    print(f"\n\n(Full response generated in {end_time - start_time:.2f} seconds)")
    full_response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    return full_response.replace(SPECIAL_STOP_TOKEN, "").strip()

def heal_and_reprint_if_needed(raw_response: str):
    """
    Checks the response for closing tags. If any are missing, it prints a healed version.
    """
    was_healed = False
    healed_response = raw_response

    # Heal missing </thinking> tag
    if "<thinking>" in healed_response and "</thinking>" not in healed_response:
        answer_pos = healed_response.find("<answer>")
        if answer_pos != -1:
            healed_response = healed_response[:answer_pos] + "</thinking>\n\n" + healed_response[answer_pos:]
            was_healed = True

    # Heal missing </answer> tag
    if "<answer>" in healed_response and "</answer>" not in healed_response:
        healed_response += "\n</answer>"
        was_healed = True

    # ONLY if we made a change, print the corrected block
    if was_healed:
        print("\n" + "="*40 + " HEALED RESPONSE " + "="*40)
        print("[SYSTEM] Original output was malformed. Displaying corrected version:")
        print("-" * 96)
        print(healed_response)
        print("="*96)

# ==============================================================================
#  STEP 3: MAIN EXECUTION LOOP
# ==============================================================================
if __name__ == "__main__":
    model, tokenizer = load_model(BASE_MODEL_PATH, LORA_ADAPTER_PATH)

    print("\n\n✅ Model is ready. Enter your code prompt below.")
    print("   Type 'exit' or 'quit' to close the script.")
    print("-" * 50)

    try:
        while True:
            user_input = input("\n>> Prompt: ")
            if user_input.lower() in ["exit", "quit"]:
                break

            print("\n🤖 Model Response (Streaming):\n" + "-"*30)
            raw_response = chat_with_model(model, tokenizer, user_input)

            # After streaming, silently check and heal the response if necessary
            heal_and_reprint_if_needed(raw_response)

    except KeyboardInterrupt:1Q
        print("\n\nExiting...")

    print("\nInference script finished.")

Loading base model...
==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 11.629 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Applying LoRA adapter from: /home/aurduinonucleo/gemma-grpo-project/Section-D/Universal-Code-Master/final_model
Optimizing model for inference...


✅ Model is ready. Enter your code prompt below.
   Type 'exit' or 'quit' to close the script.
--------------------------------------------------

🤖 Model Response (Streaming):
------------------------------
<thinking>
- The task is to write a Python function that prints "Hello".
- Two possible approaches are: 
  1. Using a simple print statement.
  2. Usin